# Quantify block diversity

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go

import altair as alt
from itertools import combinations
import numpy as np
import pypangraph as pp
from Bio import Phylo, SeqIO, pairwise2
from Bio import Align
import string

import scipy.cluster.hierarchy as sch

/opt/homebrew/Caskroom/miniforge/base/envs/junctions/lib/python3.13/site-packages/Bio/pairwise2.py:278: BiopythonDeprecationWarning: Bio.pairwise2 has been deprecated, and we intend to remove it in a future release of Biopython. As an alternative, please consider using Bio.Align.PairwiseAligner as a replacement, and contact the Biopython developers if you still need the Bio.pairwise2 module.
  warnings.warn(


In [2]:
_fname = f"../config/junction_stats.csv"
jdf = pd.read_csv(_fname)
jdf

,edge,n_iso,n_blocks,has_dupl,n_categories,majority_category,singleton,cat_entropy,n_nodes,min_length,max_length,mean_length,n_all_cores,core_left_length,core_right_length,transitive,nonempty_acc_len,nonempty_freq,pangenome_len
0,HUTOPWFGVH_f__WJBYSSJHSE_f,222,5,False,3,218,False,0.100354,11,18177,18954,18190.108108,218,16280.0,1699.0,False,727.5,0.018018,1684.0
1,IZQZNHJQRQ_f__ZVPGGEJIIF_f,222,3,False,2,221,True,0.028831,5,11740,12517,11743.500000,221,1530.0,10210.0,False,777.0,0.004505,768.0
2,JNLRGIQXOF_f__YQUHGDANHE_f,222,3,False,2,221,True,0.028831,5,28035,29472,28041.472973,221,23758.0,4277.0,False,1437.0,0.004505,1433.0
3,JMOMDSHCBS_r__SPDPCYMYDN_r,222,3,False,2,220,False,0.051397,5,7366,8143,7373.000000,220,2781.0,4585.0,False,777.0,0.009009,768.0
4,JKRDVEYDGL_f__OTPJRRWJRK_f,222,3,False,2,221,True,0.028831,5,10058,48233,10229.959459,221,7295.0,2763.0,False,38175.0,0.004505,37225.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543,GPXHVRRZLC_f__KYQOKYBCOW_f,2,12,False,2,1,True,0.693147,18,65788,105315,85551.500000,0,18606.0,29024.0,False,34255.5,1.000000,100857.0
544,EOBHADSLFU_f__SKPHAXSFLS_r,2,1,False,1,2,False,0.000000,1,7231,7231,7231.000000,2,7231.0,7231.0,True,NaN,0.000000,3298.0
545,XXIWNZXZTK_f__ZLAJFQLBFQ_r,2,3,False,2,1,True,0.693147,6,83630,83630,83630.000000,2,31644.0,48921.0,False,NaN,0.000000,73408.0
546,YUOECYBHUS_f__ZTHKZYHPIX_f,2,20,True,2,1,True,0.693147,35,66710,186048,126379.000000,0,60232.0,5710.0,False,60437.0,1.000000,160404.0
